In [ ]:
#!pip install git+https://github.com/chaofengc/IQA-PyTorch.git

In [ ]:
import pyiqa  #ipykernel
import torch
import os
import cv2
import torch
import os
from tqdm import tqdm
# list all available metrics
print(pyiqa.list_models())

device = torch.device("cuda")

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# create metric with default setting
metric = pyiqa.create_metric('hyperiqa', device=device)
print(metric.lower_better) # True or False

In [ ]:
import torch
import cv2
import os
import numpy as np
from tqdm import tqdm

# Assuming 'pyiqa' is imported properly, as it's used to create the metric

device = torch.device("dbcnn") if torch.cuda.is_available() else torch.device("cpu")

# create metric with default setting
metric = pyiqa.create_metric('dbcnn', device=device)
#metric = metric(input, task_='quality') ##only_for_qalign
scores = []
syn_images_dir = "path/to/images"  # Corrected directory name
image_paths = [os.path.join(syn_images_dir, img) for img in os.listdir(syn_images_dir)]  # Corrected variable name
# Total number of samples
total_samples = len(image_paths)

# Use tqdm to show progress bar
with tqdm(total=total_samples, desc="Processing images") as pbar:
    # Iterate through image paths
    for img_path in image_paths:
        # Read and resize image to 640x640
        image = cv2.imread(img_path)
        if image is None or image.size == 0:  # Check if the image is empty or has no pixels
            print(f"Skipping empty file: {img_path}")
            continue
        
        # Check if the image has non-zero variance of pixel values
        if np.var(image) == 0:
            print(f"Skipping image with constant pixel values: {img_path}")
            continue
        
        image = cv2.resize(image, (640, 640)) # Corrected image size
        # Convert image to RGB
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) 
        # Convert image to tensor and permute dimensions
        image = torch.from_numpy(image).permute(2, 0, 1).float().to(device)  # Moved to device
        # Compute quality score
        score = metric(image.unsqueeze(0))  # Metric expects batch dimension
        # Append score to list
        scores.append(score.item())  # Extracting scalar value
        
        # Update progress bar
        pbar.update(1)

# Save scores to txt file
with open("dbcnn_scores_synthetic_whole_dataset.txt", "w") as file:
    for s in scores:
        file.write(f"{s}\n")

print("Scores saved to brisque_scores_synthetic_whole_dataset.txt")
